<a href="https://colab.research.google.com/github/alisterpage/CHEM3580-Jupyter-Notebooks/blob/main/twisting_carbenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework - Forcefield Parameter Development: U$_\text{oop}$

For this homework exercise, we will use quantum chemistry to develop the forcefield parameters for the out-of-plane bending term ($U_\text{oop}(\chi$)) for two molecules - the methylene radical, and formaldehyde. 
 

**Run the code cell below to setup the environment and software you need for this exercise.** (It may take a minute or so to run...)


In [ ]:
#@title
!pip install pyscf geometric py3Dmol fortecubeview pythreejs -q 2>&1 >/dev/null
import pyscf
from pyscf import gto, scf, tools, lo, lib, dft
#from pyscf.geomopt.berny_solver import optimize
from pyscf.geomopt.geometric_solver import optimize
from geometric.molecule import Molecule
import py3Dmol
import fortecubeview
from google.colab import output
output.enable_custom_widget_manager()
import numpy as np
from __future__ import print_function
import sys

### Part 1: Methylene Radical 

The code cells below will optimise the geometry of the methylene radical with the H-H-H-C dihedral angle $\chi$ frozen at the value you specify. The code will use density functional theory (B3LYP/6-31G(d)), which you have used previously in CHEM2410. 

Use this code to calculate the constrained energy of the methylene radical as a function of $\chi$, for several different $\chi$ values (consider a range of 0 - 50 degrees). Ensure that the optimised χ value (reported at the end of the calculation) is approximately the same as the value you fixed it at, and inspect the optimised structure to ensure that it is reasonable. Repeat the calculation as many times as you think necessary to get a reliable correlation between the optimised energy and the dihedral angle $\chi$.



**Enter the value of the dihedral angle you want in the text box below.**

In [ ]:
#@title
import ipywidgets as widgets

dihedral_textbox = widgets.FloatText(
    value=0.0,
    description='Dihedral Angle (degrees):',
    disabled=False
)

display(dihedral_textbox)

# Get the value entered by the user in the widget and store it as a float variable called dihedral


**Run the next code cell to do the B3LYP/6-31G(d) geometry optimisation**

(You can ignore the output from this cell - the cell below summarises the key results for you)

In [ ]:
#@title
#define molecule
initial_dihedral = dihedral_textbox.value
mol = gto.Mole()
mol.atom = '''
    H
    H 1 1.8
    H 2 1.8 1 60.0
    C 1 1.1 2 35.2 3 {}
'''.format(initial_dihedral)

#define model chemistry & calculation 
mol.basis = '6-31g(d)'
mol.spin=1;
mol.build();
mf = dft.ROKS(mol)
mf.xc = 'b3lyp'
mf = mf.newton()
mf.kernel()

# Optimize the geometry
text = "$freeze\ndihedral 1 2 3 4\n"
with open("constraints.txt", "w") as f:
    f.write(text)
params = {"constraints": "constraints.txt",}
mol_eq = optimize(mf, **params)

#print(mol_eq.atom_coords())

# Get the optimized energy
mf = dft.ROKS(mol_eq)
mf.xc = 'b3lyp'
mf = mf.newton()
mf.kernel()

# Create a temporary XYZ file with the optimized coordinates
symbols = [atom[0] for atom in mol_eq.atom];
coords = mol_eq.atom_coords();
with open("temp.xyz", "w") as f:
    f.write("%d\n" % len(symbols))
    f.write("Generated by PySCF and Geometric\n")
    for i in range(len(symbols)):
        f.write("%s %f %f %f\n" % (symbols[i], coords[i,0], coords[i,1], coords[i,2]))

# Convert the PySCF object to a `Molecule` object using the XYZ file and store dihedral angle.
mol_geom = Molecule("temp.xyz");
dihedral=Molecule.measure_dihedrals(mol_geom,0,1,2,3);

**Run the next code cell to summarise the calculation and show the geometry of the molecule**

In [ ]:
#@title
#print summary
print('Initial dihedral before geometry optimisation:', initial_dihedral,"degrees")
print('Final dihedral after geometry optimisation:', abs(dihedral[0]), 'degrees')
print('Final energy after geometry optimisation:', mf.e_tot,"Hartree")

# get the atomic positions & write PDB file
atom_pos = mol.atom_coords()
atom_symbol = mol._atom

with open('mol.xyz', 'w') as f:
    f.write(f'{len(mol._atom)}\n')
    f.write('opt geom\n')
    for i, (symbol, pos) in enumerate(zip(atom_symbol, atom_pos)):
        f.write(f'{symbol[0]}  {pos[0]:>7.3f}{pos[1]:>7.3f}{pos[2]:>7.3f}\n')


view = py3Dmol.view()
view.addModel(open('mol.xyz', 'r').read(),'xyz')
view.setBackgroundColor('#CFB691')
view.setStyle({'sphere': {'radius': 0.5}})
view.zoomTo()
view.show()


### Part 2: Formaldehyde

The code cells below will optimise the geometry of formaldehyde with the O-H-H-C dihedral angle $\chi$ frozen at the value you specify. The code will use density functional theory (B3LYP/6-31G(d)), which you have used previously in CHEM2410. _This is the same level of theory used for methylene, above_.

Use this code to calculate the constrained energy of formaldehyde as a function of $\chi$, for several different $\chi$ values (consider a range of 0 - 50 degrees). Ensure that the optimised χ value (reported at the end of the calculation) is approximately the same as the value you fixed it at, and inspect the optimised structure to ensure that it is reasonable. Repeat the calculation as many times as you think necessary to get a reliable correlation between the optimised energy and the dihedral angle $\chi$.



**Enter the value of the dihedral angle you want in the text box below.**

In [ ]:
#@title
import ipywidgets as widgets

dihedral_textbox = widgets.FloatText(
    value=0.0,
    description='Dihedral Angle (degrees):',
    disabled=False
)

display(dihedral_textbox)

# Get the value entered by the user in the widget and store it as a float variable called dihedral


**Run the next code cell to do the B3LYP/6-31G(d) geometry optimisation**

(You can ignore the output from this cell - the cell below summarises the key results for you)

In [ ]:
#@title
#define molecule
initial_dihedral = dihedral_textbox.value
mol = gto.Mole()
mol.atom = '''
    O
    H 1 2.03
    H 2 1.87 1 62.0
    C 1 1.2 2 27.5 3 {}
'''.format(initial_dihedral)

#define model chemistry & calculation 
mol.basis = '6-31g(d)'
mol.spin=0;
mol.build();
mf = dft.RKS(mol)
mf.xc = 'b3lyp'
mf = mf.newton()
mf.kernel()

# Optimize the geometry
text = "$freeze\ndihedral 1 2 3 4\n"
with open("constraints.txt", "w") as f:
    f.write(text)
params = {"constraints": "constraints.txt",}
mol_eq = optimize(mf, **params)

#print(mol_eq.atom_coords())

# Get the optimized energy
mf = dft.RKS(mol_eq)
mf.xc = 'b3lyp'
mf = mf.newton()
mf.kernel()

# Create a temporary XYZ file with the optimized coordinates
symbols = [atom[0] for atom in mol_eq.atom];
coords = mol_eq.atom_coords();
with open("temp.xyz", "w") as f:
    f.write("%d\n" % len(symbols))
    f.write("Generated by PySCF and Geometric\n")
    for i in range(len(symbols)):
        f.write("%s %f %f %f\n" % (symbols[i], coords[i,0], coords[i,1], coords[i,2]))

# Convert the PySCF object to a `Molecule` object using the XYZ file and store dihedral angle.
mol_geom = Molecule("temp.xyz");
dihedral=Molecule.measure_dihedrals(mol_geom,0,1,2,3);

**Run the next code cell to summarise the calculation and show the geometry of the molecule**

**NOTE: The 'Final dihedral' value is the one you need to use in determining $k_c$.**

In [ ]:
#@title
#print summary
print('Initial dihedral before geometry optimisation:', initial_dihedral,"degrees")
print('Final dihedral after geometry optimisation:', abs(dihedral[0]), 'degrees')
print('Final energy after geometry optimisation:', mf.e_tot,"Hartree")

# get the atomic positions & write PDB file
atom_pos = mol.atom_coords()
atom_symbol = mol._atom

with open('mol.xyz', 'w') as f:
    f.write(f'{len(mol._atom)}\n')
    f.write('opt geom\n')
    for i, (symbol, pos) in enumerate(zip(atom_symbol, atom_pos)):
        f.write(f'{symbol[0]}  {pos[0]:>7.3f}{pos[1]:>7.3f}{pos[2]:>7.3f}\n')


view = py3Dmol.view()
view.addModel(open('mol.xyz', 'r').read(),'xyz')
view.setBackgroundColor('#CFB691')
view.setStyle({'sphere': {'radius': 0.5}})
view.zoomTo()
view.show()


### Questions 

1. Use your data to estimate the out-of-plane bending force constant $k^c$ for the out-of-plane bending potentials in the methylene radical and formaldeyde. (It is easiest to do this analysis in Microsoft Excel.)
1. Both of these molecules have equivalent structure and a central sp$^2$-hybridised carbon atom. Why do you think the $k^c$ values for these two potentials are so different? Explain your answer in terms of the electronic structure and bonding in these molecules.

**Submit your homework as a single MS excel file. You will be assessed on (1) the accuracy of your force constant value, and its $\pm$ margin of error, and (2) your answer to question 2, above.**